In [1]:
import json
import itertools
import zipfile
from datetime import date
from pathlib import Path

import cv2
import rasterio
import utm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub import pixel_to_utm, utm_to_pixel

## Collect the satellite imagery data

In [2]:
# Using pathlib for handling paths which is convientent way. And set path to ../data folder from current context.
data = Path("../data/")

In [20]:
# Set your user and password.
with open("../secrets/sentinel_api_credentials.json", "r") as file:
    credentials = json.load(file)

api = SentinelAPI(credentials["username"], credentials["password"], "https://scihub.copernicus.eu/dhus")

shape = geojson_to_wkt(read_geojson(data / "cameroon.geojson"))

# Try to make the data be between 2 months because image may not be captured by satellite for small intervals.
images = api.query(
    shape,
    date=(date(2023, 1, 1), date(2023, 3, 1)),
    platformname="Sentinel-2",
    processinglevel = "Level-2A",
    cloudcoverpercentage=(0, 30)
)

images_df = api.to_dataframe(images)

In [21]:
images_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 386009ee-2a37-4b58-9f55-03f0d5cea946 to 39f7c5d8-2df1-4b4d-ae05-63641f364fa2
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   title                        34 non-null     object        
 1   link                         34 non-null     object        
 2   link_alternative             34 non-null     object        
 3   link_icon                    34 non-null     object        
 4   summary                      34 non-null     object        
 5   ondemand                     34 non-null     object        
 6   generationdate               34 non-null     datetime64[ns]
 7   beginposition                34 non-null     datetime64[ns]
 8   endposition                  34 non-null     datetime64[ns]
 9   ingestiondate                34 non-null     datetime64[ns]
 10  orbitnumber                  34 non-null     int

In [22]:
images_df.head()

,title,link,link_alternative,link_icon,summary,ondemand,generationdate,beginposition,endposition,ingestiondate,...,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,datastripidentifier,granuleidentifier,identifier,uuid
386009ee-2a37-4b58-9f55-03f0d5cea946,S2A_MSIL2A_20230220T093031_N0509_R136_T32NNL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:09:35.637,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NNL_2...,386009ee-2a37-4b58-9f55-03f0d5cea946
e3eea048-79fd-4f9d-8c3c-450998853276,S2A_MSIL2A_20230220T093031_N0509_R136_T32NNK_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:09:24.695,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NNK_2...,e3eea048-79fd-4f9d-8c3c-450998853276
8daddb67-7ad8-4838-a460-dab8ab3a4dda,S2A_MSIL2A_20230220T093031_N0509_R136_T32NPL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:08:32.950,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NPL_2...,8daddb67-7ad8-4838-a460-dab8ab3a4dda
2153894d-6887-45bf-a30c-4edeed334f5e,S2A_MSIL2A_20230220T093031_N0509_R136_T32NQL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:06:24.627,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NQL_2...,2153894d-6887-45bf-a30c-4edeed334f5e
5dca32b8-24de-4a33-8bd8-f1acf34af6ca,S2A_MSIL2A_20230220T093031_N0509_R136_T32NRL_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2023-02-20T09:30:31.025Z, Instrument: MS...",false,2023-02-20 14:34:08,2023-02-20 09:30:31.025,2023-02-20 09:30:31.025,2023-02-20 16:02:50.587,...,GS2A_20230220T093031_040030_N05.09,S2MSI2A,2015-028A,DESCENDING,Sentinel-2A,Level-2A,S2A_OPER_MSI_L2A_DS_2APS_20230220T143408_S2023...,S2A_OPER_MSI_L2A_TL_2APS_20230220T143408_A0400...,S2A_MSIL2A_20230220T093031_N0509_R136_T32NRL_2...,5dca32b8-24de-4a33-8bd8-f1acf34af6ca


In [43]:
# Randomaly select a uuid.
uuids = images_df["uuid"].sample().iloc[0]
uuids

'818c53ea-c39d-44ce-8bae-adf27769576c'

In [44]:
details = api.download(uuids, directory_path=data)
details

MD5 checksumming:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

{'id': '818c53ea-c39d-44ce-8bae-adf27769576c',
 'title': 'S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128',
 'size': 1162740424,
 'md5': 'f420e417fae5a87ef685627212a7b3b2',
 'date': datetime.datetime(2023, 1, 6, 9, 32, 59, 24000),
 'footprint': 'POLYGON((10.80472775247355 5.425531020312141,11.795064553781264 5.42176328831644,11.790920736919428 4.429629774710796,10.80204968546757 4.432705100697929,10.80472775247355 5.425531020312141))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('818c53ea-c39d-44ce-8bae-adf27769576c')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2023, 1, 6, 15, 5, 4, 431000),
 'Ingestion Date': datetime.datetime(2023, 1, 6, 15, 4, 38, 318000),
 'quicklook_url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('818c53ea-c39d-44ce-8bae-adf27769576c')/Products('Quicklook')/$value",
 'path': '../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.zip',
 'downloaded_bytes': 1162740424}

In [45]:
# Extract the zip file and delete it.
if Path(details["path"]).exists():
    with zipfile.ZipFile(details["path"], 'r') as zip_ref:
        print("Extracting and deleting zipfile.")
        zip_ref.extractall(data)

    Path(details["path"]).unlink(missing_ok=True)
else:
    print("Zip file does not exist.")

Extracting and deleting zipfile.


In [3]:
## Still working on it.
def get_band(image_folder, band, resolution=10):

    # subfolder = [f for f in os.listdir(image_folder + "/GRANULE") if f[0]  == "L"][0]
    sub_folder = [f for f in (image_folder / "GRANULE").iterdir() if f.name.startswith("L")][0]

    # image_folder_path  = image_folder / "GRANULE" / subfolder / "IMG_DATA" / f"R{resolution}m"
    image_folder  = sub_folder / "IMG_DATA" / f"R{10}m"

    # image_files = [im for im in os.listdir(image_folder_path) if im[-4:] == ".jp2"]
    image_files = [im for im in image_folder.iterdir() if im.suffix == ".jp2"]
    selected_file = [im for im in image_files if im.name.split("_")[2] == band][0]
    
    with rasterio.open(selected_file) as file:
        img = file.read(1)
            
    return img

In [4]:
data_1_folder = data / "S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE"

sub_folder = [f for f in (data_1_folder / "GRANULE").iterdir() if f.name.startswith("L")][0]
sub_folder

PosixPath('../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402')

In [5]:
image_folder  = sub_folder / "IMG_DATA" / f"R{10}m"
image_folder

PosixPath('../data/S2B_MSIL2A_20230106T093259_N0509_R136_T32NQL_20230106T122128.SAFE/GRANULE/L2A_T32NQL_A030478_20230106T094402/IMG_DATA/R10m')

In [6]:
# check the image path's in image_folder_path of R10m
for path in image_folder.iterdir():
    print(path.name)

T32NQL_20230106T093259_AOT_10m.jp2
T32NQL_20230106T093259_WVP_10m.jp2
T32NQL_20230106T093259_B03_10m.jp2
T32NQL_20230106T093259_B04_10m.jp2
T32NQL_20230106T093259_TCI_10m.jp2
T32NQL_20230106T093259_B02_10m.jp2
T32NQL_20230106T093259_B08_10m.jp2


## Understand which spectral band to use and manage quantization.

Quantization is the process of converting continuous data (such as radiance values) into discrete values (such as digital numbers) that can be stored and processed by a computer. This is necessary because computers can't store or process continuous data, so it needs to be discretized into manageable chunks.

In [7]:
band_dict = {}

for band in ["B02", "B03", "B04", "B08"]:
    band_dict[band] = get_band(data_1_folder, band, 10)

band_dict

{'B02': array([[1384, 1394, 1356, ..., 1170, 1187, 1244],
        [1334, 1344, 1322, ..., 1188, 1222, 1229],
        [1308, 1280, 1254, ..., 1176, 1168, 1216],
        ...,
        [1233, 1229, 1247, ..., 1173, 1205, 1220],
        [1240, 1298, 1288, ..., 1210, 1176, 1190],
        [1274, 1292, 1267, ..., 1171, 1171, 1168]], dtype=uint16),
 'B03': array([[1478, 1514, 1454, ..., 1312, 1403, 1424],
        [1422, 1455, 1450, ..., 1273, 1380, 1461],
        [1417, 1415, 1418, ..., 1327, 1326, 1416],
        ...,
        [1480, 1456, 1452, ..., 1368, 1416, 1474],
        [1414, 1435, 1472, ..., 1356, 1385, 1431],
        [1452, 1434, 1430, ..., 1372, 1370, 1420]], dtype=uint16),
 'B04': array([[1616, 1684, 1588, ..., 1302, 1304, 1356],
        [1614, 1618, 1595, ..., 1304, 1280, 1380],
        [1592, 1476, 1447, ..., 1311, 1266, 1363],
        ...,
        [1321, 1330, 1354, ..., 1301, 1356, 1326],
        [1333, 1332, 1324, ..., 1290, 1293, 1342],
        [1352, 1374, 1372, ..., 1309, 134

In [8]:
# Recreating an RGB image 
img = cv2.merge((band_dict["B04"], band_dict["B03"], band_dict["B02"]))
plt.imshow(img)

: 

: 